In [3]:
import os
import pandas
import cv2
import numpy

In [5]:
class selective_tracker():

    data = pandas.read_csv('/Users/akshitshishodia/intern/tracker/labels_my-project-name_2023-06-27-09-45-54.csv')
    pos = 0
    neg = 0
    def __init__(self) -> None:
        pass
    def calculate_iou(box_a,box_b):
        xA = max(box_a[0],box_b[0])
        yA = max(box_a[1],box_b[1])
        xB = min(box_a[2],box_b[2])
        yB = min(box_a[3],box_b[3])
        inter_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)
        pred = (box_a[2]-box_a[0]+1)*(box_a[3]-box_a[1]+1)
        ground_truth = (box_b[2]-box_b[0]+1)*(box_b[3]-box_b[1]+1)
        iou = inter_area / float(pred + ground_truth - inter_area)
        return iou
    @classmethod
    def annotation_extracter(cls,path):
        totalpos = 0
        totalneg = 0
        image_path = [file for file in os.listdir(path=path)]
        gt_box = list()
        for image_loc in image_path:
            row = cls.data[cls.data['image_name']== image_loc]
            w = int(row.image_width)
            h = int(row.image_height)
            x_min = int(row.bbox_x)
            y_min = int(row.bbox_y)
            x_max= int(row.bbox_x)+int(row.bbox_width)
            y_max = int(row.bbox_y)+int(row.bbox_height)

            x_min = max(0,x_min)
            y_min = max(0,y_min)
            x_max = min(w,x_max)
            y_max = min(h,y_max)

            gt_box.append((x_min,y_min,x_max,y_max))
            img = cv2.imread(os.path.join(path,image_loc))
            ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
            ss.setBaseImage(img)
            ss.switchToSelectiveSearchQuality()
            rects = ss.process()
            proposedRegions = list()
            for x,y,w,h in rects:
                pos_roi = 0
                neg_roi = 0
                proposedRegions.append((x,y,x+w,y+h))
                for proposedRect in proposedRegions[:2000]:
                    (x1,y1,x2,y2) = proposedRect
                    for gt in gt_box:
                        IOU = cls.calculate_iou(gt,proposedRect)
                        roi = None
                        OutputPath = None
                        (gx1,gy1,gx2,gy2) = gt

                        if IOU > 0.7 and cls.pos <= 2000:
                            roi = img[y1:y2,x1:x2]
                            loc = str(totalpos)+'.png'
                            OutputPath = os.path.join('/Users/akshitshishodia/intern/tracker/refined/good',loc)
                            totalpos+=1
                            cls.pos+=1                            
                        fullOverlap = x1 >= gx1
                        fullOverlap = fullOverlap and y1 >= gy1
                        fullOverlap = fullOverlap and x2 <= gx2
                        fullOverlap = fullOverlap and y2 <= gy2

                        if not fullOverlap and IOU < 0.05 and cls.neg<=2000:
                            roi = img[y1:y2,x1:x2]
                            loc = str(totalneg)+'.png'
                            OutputPath = os.path.join('/Users/akshitshishodia/intern/tracker/refined/good/bad',loc)
                            totalneg+=1
                            cls.neg+=1
                        
                        if roi is not None and OutputPath is not None:
                            roi = cv2.resize(roi,(224,224),cv2.INTER_CUBIC)
                            cv2.imwrite(OutputPath,roi)










            
            





        
        

test_obj = selective_tracker()
test_obj.annotation_extracter('/Users/akshitshishodia/intern/tracker/dataset')


KeyboardInterrupt: 

In [26]:
data = pandas.read_csv('/Users/akshitshishodia/intern/tracker/labels_my-project-name_2023-06-27-09-45-54.csv')
data.head()
# for file in os.listdir(path='/Users/akshitshishodia/intern/tracker/dataset'):
#     display(file)
img = cv2.imread('/Users/akshitshishodia/intern/tracker/dataset/frame540.jpg')
row = data[data['image_name']=='frame540.jpg']
x_min = int(row.bbox_x)
y_max = int(row.bbox_y)
x_max= int(row.bbox_x)+int(row.bbox_width)
y_min = int(row.bbox_y)+int(row.bbox_height)
img = cv2.rectangle(img,(x_min,y_min),(x_max,y_max),(0,255,0),5)
cv2.imshow('frame',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
# s = int(i.bbox_height)
# data.head()